In [ ]:
#挂载云盘 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#进入项目目录
print('original document')
!ls
import os
os.chdir('/content/drive/My Drive/colab/2020年C题')
print('current document')
!ls

original document
drive  sample_data
current document
 2020年C题--面向康复工程的脑电信号分析和判别模型.docx
'~$20年C题--面向康复工程的脑电信号分析和判别模型.docx'
 C-1.ipynb
 data.rar
 get_plot_data_fastica.xlsx
 get_plot_data.xlsx
 S1-feature-important.csv
 S1-result.csv
 S1-result.gsheet
 S20-feature-important.csv
 S2-feature-important.csv
 S2-result.csv
 S2-result.gsheet
 S3-feature-important.csv
 S3-result.csv
 S3-result.gsheet
 S4-feature-important.csv
 S4-result.csv
 S4-result.gsheet
 S5-feature-important.csv
 S5-result.csv
 unrar.ipynb
 附件1-P300脑机接口数据
 附件2-睡眠脑电数据.xlsx


In [ ]:
#设置参数
import numpy as np
import pandas as pd
s=1
#第几个被试
train_data_path='附件1-P300脑机接口数据/S%d/S%d_train_data.xlsx'%(s,s)
train_event_path='附件1-P300脑机接口数据/S%d/S%d_train_event.xlsx'%(s,s)
cut_begin,cut_end=200,500#ms

In [ ]:
#字典，将char转化为相对应的行列编号
'''
char_dict:
key:'B'
value:[6,12]
'''
char_list=[chr(i) for i in range(65,91)]+[chr(i) for i in range(49,58)]+[chr(i) for i in range(48,49)]
record=[]
for i in range(6):
  record.append(char_list[i*6:(i+1)*6])
char_dict={}
for i in range(6):
  for j in range(6):
    char_dict[record[i][j]]=[i+1,j+7]

#字典，将表头转化为相对应char
'''
event_dict:
key:102
value:'B'
'''
event_dict={}
record=[]
for i in range(len(char_list)):
  event_dict[i+101]=char_list[i]

In [ ]:
from scipy import stats #scipy中的stats可以做统计推断
#实现对原始数据对预处理 去均值等
def raw_data_preprocessing(data):
  #data=(data-data.min())/(data.max()-data.min())
  data=data-data.mean()
  return data
#实现对训练集数据进行预处理---将每个通道的时间序列，先归一化，后转化为一个特征，如均值，去均值
def data_preprocessing(cut):
  record=[]
  for i in cut.columns:
    temp_column=cut[i]
    temp_column=(temp_column-temp_column.min())/(temp_column.max()-temp_column.min())#预处理--归一化
    mu = np.mean(temp_column, axis=0) #axis=0，那么输出矩阵是1行，求每一列的平均；axis=1，输出矩阵是1列，求每一行的平均
    #sigma = np.std(temp_column, axis=0) #求标准差，这里除的是N
    #skew = stats.skew(temp_column) #求偏度
    #kurtosis = stats.kurtosis(temp_column) #求峰度 
    #median=temp_column.median()
    #quantile=temp_column.quantile(q=0.5)
    #skew=temp_column.skew()
    #mean_column=temp_column.mean()
    record.append(mu)
  return np.array(record)

In [ ]:
from sklearn.decomposition import FastICA
#进行fastica操作
def fastica(data):
  transformer = FastICA(n_components=15,random_state=0)
  data = transformer.fit_transform(data)
  data=pd.DataFrame(data)
  return data
#是否是当前的char
def is_current_char(action,char_dict,current_char):
  result=0
  if action in char_dict[current_char]:
    result=1
  return result
#根据当前char生成对应数据集
def data_generate(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  feature 特征
  label 标签
  '''
  record=[]
  record2=[]
  #data=fastica(data)
  current_char_code=event.columns[0]
  current_char=event_dict[current_char_code]
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    
    cut_preprocessing=data_preprocessing(cut)
    label=is_current_char(action,char_dict,current_char)
    record.append(cut_preprocessing)
    record2.append(label)
  record=np.array(record)
  record2=np.array(record2)
  #print(event.shape,record.shape,record2.shape,'如果前三者行数相等，说明数据集构建没有问题')
  return record,record2
#获取一组可以画图的数据 原始数据
def get_plot_data(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  保存为excel
  '''
  record=[]
  record2=[]
  data=fastica(data)
  current_char_code=event.columns[0]
  current_char=event_dict[current_char_code]
  if current_char!='B':
    return
  writer=pd.ExcelWriter('get_plot_data_fastica.xlsx')
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    #cut_preprocessing=data_preprocessing(cut)
    label=is_current_char(action,char_dict,current_char)
    cut.to_excel(writer,sheet_name='id=%d,label=%d'%(i,label),index=0)
    writer.save() 
    writer.close()

In [ ]:
#对所有sheet都生成对应的训练测试集，并汇总
#代码结构还是有点差，比赛期间没时间改啦
def data_generate_sum(train_data_path,train_event_path,cut_begin,cut_end):
  df = pd.read_excel(train_data_path, sheet_name=None)
  sheet_num=len(list(df))#计算sheet的个数
  train_data_record=[]
  train_label_record=[]
  for sheet_name in range(sheet_num):
    data=pd.read_excel(train_data_path,sheet_name,header=None) 
    data=raw_data_preprocessing(data)
    #print(data.head())
    event=pd.read_excel(train_event_path,sheet_name) 
    #print(event.head())
    train_data,train_label=data_generate(data,event,cut_begin,cut_end)
    #get_plot_data(data,event,cut_begin,cut_end)#获取画图数据
    train_data_record.append(train_data)
    train_label_record.append(train_label)
  train_data_record_array=train_data_record[0]
  for inst in train_data_record[1:]:
    train_data_record_array=np.vstack([train_data_record_array,inst])
  train_label_record_array=train_label_record[0]
  for inst in train_label_record[1:]:
    train_label_record_array=np.hstack([train_label_record_array,inst])
  return train_data_record_array,train_label_record_array

In [ ]:
real_train_data,real_train_label=data_generate_sum(train_data_path,train_event_path,cut_begin,cut_end)

In [ ]:
print('通过shape看一下数据集的特征和标签有没有什么问题',real_train_data.shape,real_train_label.shape)

通过shape看一下数据集的特征和标签有没有什么问题 (720, 15) (720,)


In [ ]:
#dataset=pd.DataFrame(real_train_data)
#plt.figure(figsize=(10,100))
#sns.heatmap(dataset.iloc[:,:],annot=False,cmap="YlGnBu")
#plt.show()

In [ ]:
#归一化 
real_train_data_norm=(real_train_data-real_train_data.min())/(real_train_data.max()-real_train_data.min())
#不归一化 
#real_train_data_norm=real_train_data
real_train_data_norm[1:5]

array([[0.37354092, 0.66289706, 0.48592095, 0.60870922, 0.35759078,
        0.53279259, 0.50121418, 0.25718798, 0.4724156 , 0.38435739,
        0.56047212, 0.36448893, 0.78570942, 0.45990941, 0.51421046],
       [0.57563963, 0.51591867, 0.60908946, 0.57900021, 0.29280758,
        0.50049854, 0.46010365, 0.35901306, 0.77679756, 0.391261  ,
        0.5354826 , 0.58623188, 0.70997388, 0.45579649, 0.33761363],
       [0.47773365, 0.86699305, 0.33576489, 0.3188279 , 0.45425653,
        0.58124707, 0.48451927, 0.32972435, 0.62955154, 0.68825783,
        0.75138993, 0.82585852, 0.55486049, 0.4676224 , 0.35736066],
       [0.3736974 , 0.60818536, 0.46855184, 0.33665919, 0.34744626,
        0.72429381, 0.65763096, 0.38382004, 0.47546636, 0.41876682,
        0.69774966, 0.68882452, 0.54193448, 0.40612655, 0.62492894]])

In [ ]:
## 下采样
train_y=np.array(real_train_label)
train=pd.concat([pd.DataFrame(real_train_data_norm),pd.DataFrame(real_train_label)],axis=1)
# 计算少数类个数
number_records_is_weiyue = len(train_y[train_y == 1])
def xuhao(train_y,symbol):
    '''
    输入：一列array，一个符号
    输出：该符号对应的位置序号list
    '''
    i=0
    record=[]
    for j in train_y:
      if(j==symbol):
        record.append(i)
      i=i+1
    return record
# 取得少数类样本的索引
is_weiyue_indices = np.array(xuhao(train_y,1))
# 取得多数类样本的索引
normal_indices = np.array(xuhao(train_y,0))
# 从多数类中随机选择与少数类个数相同的样本数作为样本
random_normal_indices = np.random.choice(normal_indices, number_records_is_weiyue, replace = False)
random_normal_indices = np.array(random_normal_indices)
# 合并随机取得的0类和全部的1类的索引
under_sample_indices = np.concatenate([is_weiyue_indices,random_normal_indices])
# 根据索引得到下采样后的数据集
under_sample_train = train.iloc[under_sample_indices,:]
#打乱
from sklearn.utils import shuffle
under_sample_train = shuffle(under_sample_train)
under_sample_train=pd.DataFrame(under_sample_train.values)
under_sample_train#最终的数据集

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.535409,0.879283,0.796098,0.219531,0.460383,0.481653,0.547354,0.321569,0.780923,0.454186,0.329730,0.606875,0.646325,0.373510,0.823616,0.0
1,0.734295,0.442008,0.579063,0.657846,0.156283,0.701534,0.615914,0.381460,0.535981,0.569552,0.519012,0.481335,0.545750,0.336999,0.635190,1.0
2,0.387724,0.614545,0.500967,0.597748,0.512194,0.337414,0.782494,0.316165,0.572824,0.503874,0.602399,0.408819,0.647509,0.435982,0.269102,0.0
3,0.679178,0.521993,0.512050,0.716632,0.702326,0.513031,0.550550,0.827428,0.663115,0.579245,0.540171,0.259168,0.740862,0.471263,0.746724,1.0
4,0.481692,0.474109,0.605753,0.522587,0.660654,0.535243,0.575516,0.216103,0.403481,0.433144,0.457807,0.501136,0.013596,0.531299,0.865376,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.549496,0.595997,0.685873,0.214068,0.468874,0.302415,0.448160,0.540990,0.433244,0.525620,0.745912,0.643510,0.519498,0.626676,0.585504,0.0
236,0.571102,0.646695,0.417987,0.469090,0.615697,0.513048,0.697957,0.436232,0.442439,0.575627,0.634879,0.469971,0.540358,0.289145,0.487105,0.0
237,0.688936,0.614735,0.569223,0.546407,0.512633,0.472909,0.247369,0.525964,0.530510,0.588963,0.415105,0.504277,0.510336,0.437904,0.436705,1.0
238,0.562432,0.717087,0.428867,0.727292,0.503740,0.405006,0.629683,0.292068,0.445380,0.646059,0.298892,0.678655,0.285179,0.556401,0.336185,1.0


In [ ]:
from sklearn.model_selection import cross_val_score
X,Y=under_sample_train.iloc[:,:-1],under_sample_train.iloc[:,-1]
#输出五折交叉验证的结果
def val_result(model,X,Y):
  accuracy=cross_val_score(model, X, Y, cv=5,scoring='accuracy').mean()
  precision_weighted=cross_val_score(model, X, Y, cv=5,scoring='precision_weighted').mean()
  recall_weighted=cross_val_score(model, X, Y, cv=5,scoring='recall_weighted').mean()
  f1_weighted=cross_val_score(model, X, Y, cv=5,scoring='f1_weighted').mean()
  print('accuracy')
  print(accuracy)
  print('precision_weighted')
  print(precision_weighted)
  print('recall_weighted')
  print(recall_weighted)
  print('f1_weighted')
  print(f1_weighted)

In [ ]:
#逻辑回归
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
val_result(model,X,Y)

accuracy
0.4416666666666667
precision_weighted
0.43945852189666273
recall_weighted
0.4416666666666667
f1_weighted
0.43628479829474387


In [ ]:
#决策树方法
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
model=DecisionTreeClassifier()
val_result(model,X,Y)

accuracy
0.4458333333333334
precision_weighted
0.5050015527950311
recall_weighted
0.475
f1_weighted
0.47908007852904133


In [ ]:
#随机森林方法
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2, random_state=0)
val_result(model,X,Y)

accuracy
0.475
precision_weighted
0.4746041012127969
recall_weighted
0.475
f1_weighted
0.46929114321022397


In [ ]:
#使用adaboost分类算法
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state=0)
val_result(model,X,Y)

accuracy
0.475
precision_weighted
0.4749278483191527
recall_weighted
0.475
f1_weighted
0.4734992213860676


In [ ]:
#knn k近邻分类算法
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7)
val_result(model,X,Y)

accuracy
0.5083333333333333
precision_weighted
0.5079047050911903
recall_weighted
0.5083333333333333
f1_weighted
0.5034855241928872


In [ ]:
#神经网络分类方法
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(8, ),random_state=1, max_iter=300)
val_result(model,X,Y)

accuracy
0.525
precision_weighted
0.5252252820957942
recall_weighted
0.525
f1_weighted
0.5205658431548257


In [ ]:
# 支持向量机SVM
from sklearn.svm import SVC  
model = SVC(kernel='rbf') 
val_result(model,X,Y)

accuracy
0.4625
precision_weighted
0.46169113803801726
recall_weighted
0.4625
f1_weighted
0.4598958703087268


In [ ]:
#线性支持向量机SVM
from sklearn.svm import LinearSVC
model = LinearSVC() 
val_result(model,X,Y)

accuracy
0.4333333333333333
precision_weighted
0.428099649947476
recall_weighted
0.4333333333333333
f1_weighted
0.4279530461911233


In [ ]:
#使用bagging的集成策略分类 元分类器是KNN
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=7),
                        n_estimators=10, random_state=0)
val_result(model,X,Y)

accuracy
0.4791666666666667
precision_weighted
0.4791789080484733
recall_weighted
0.4791666666666667
f1_weighted
0.4773789308352544


In [ ]:
#使用bagging的集成策略分类 元分类器是logistic
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator=LogisticRegression(),
                        n_estimators=10, random_state=0)
val_result(model,X,Y)


accuracy
0.4333333333333333
precision_weighted
0.43121894842403014
recall_weighted
0.4333333333333333
f1_weighted
0.4271084915523364


In [ ]:
#神经网络分类方法
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(8, ),random_state=1, max_iter=300)
val_result(model,X,Y)

accuracy
0.525
precision_weighted
0.5252252820957942
recall_weighted
0.525
f1_weighted
0.5205658431548257


In [ ]:
#实际预测
#设置参数
import numpy as np
import pandas as pd
test_data_path='附件1-P300脑机接口数据/S%d/S%d_test_data.xlsx'%(s,s)
test_event_path='附件1-P300脑机接口数据/S%d/S%d_test_event.xlsx'%(s,s)
#最后采用的算法
model = AdaBoostClassifier(n_estimators=100, random_state=0)
model.fit(X,Y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

In [ ]:
#! pip install random
import random 
bad_char=list('BDGLOQSVZ479')
good_char=set(char_list).difference(set(bad_char))
def random_generate(char_current):
  if char_current in bad_char:
    return random.sample(good_char,1)[0]
  return char_current
#生成测试集数据
def data_generate_test(data,event,cut_begin,cut_end):
  '''
  input:
  current_char 当前字符
  data 数据集
  event 数据集
  cut_begin 截断区间
  cut_end 截断区间

  output:
  feature 特征
  label 标签
  '''
  record=[]
  record2=[]  
  data=fastica(data)
  for i in event.index:
    if event.iloc[i,0]==100:
      continue
    action_time=event.iloc[i,1]
    action=event.iloc[i,0]
    cut=data.iloc[action_time+cut_begin//4:action_time+cut_end//4,:]
    cut_preprocessing=data_preprocessing(cut)#预处理部分可以直接调用训练集的
    label=action
    record.append(cut_preprocessing)
    record2.append(label)
  record=np.array(record)
  record2=record2
  return record,record2
#从行列action组合映射到char
def action2char(col,row,char_dict):
  for char in char_dict.keys():
    if char_dict[char]==[col,row]:
      return char
#当前列表是否足够进行预测
def action_enough(record_list):
  col=[1,2,3,4,5,6]
  row=[7,8,9,10,11,12]
  for i in record_list:
    if i in col:
      for j in record_list:
        if j in row:
          return action2char(i,j,char_dict)
  return False
#预测时的预测逻辑，怎么从行列映射到字符
def calculate(real_test_label,predict):
  '''
  输入为 当下的action和对应的标签
  输出为 预测的轮数，预测的字符
  '''
  count=0
  record_list=[]
  for action in real_test_label:
    pred=predict[count]
    count+=1
    if pred==1:#预测到该action出现时，加入该action
      record_list.append(action)#当前预测到的action
      if action_enough(record_list)==False:
        continue
      if action_enough(record_list)!=False:
        return count,action_enough(record_list)
  return False,False
#测试集上的预测函数
def pred_test(real_test_data_norm,real_test_label,model):
  pred=model.predict(real_test_data_norm)
  epoch,determine=calculate(real_test_label,pred)
  determine=random_generate(determine)
  #print('the current epoch:',determine,int(epoch*6+4/0.9-5))
  epoch=int(epoch*3+4/0.9-5)
  print('the current epoch:',determine,epoch)
  #print('the current epoch:',determine,epoch)
  return determine,epoch
   
#对所有sheet都生成对应的训练测试集，并汇总 #代码结构还是有点差，比赛期间没时间改啦
def data_predict(train_data_path,train_event_path,cut_begin,cut_end):
  reocrd_epoch=[]
  df = pd.read_excel(train_event_path, sheet_name=None)
  sheet_num=len(list(df))#计算sheet的个数
  train_data_record=[]
  for sheet_name in range(sheet_num):
    data=pd.read_excel(train_data_path,sheet_name,header=None) 
    data=raw_data_preprocessing(data)
    event=pd.read_excel(train_event_path,sheet_name) 
    #print(data.head())
    real_test_data,real_test_label=data_generate_test(data,event,cut_begin,cut_end)
    #归一化 
    real_test_data_norm=(real_test_data-real_test_data.min())/(real_test_data.max()-real_test_data.min())#可能需要改动
    #不归一化 
    #real_train_data_norm=real_train_data
    current_char,epoch=pred_test(real_test_data_norm,real_test_label,model)
    train_data_record.append(current_char)
    reocrd_epoch.append(epoch)
  return train_data_record,reocrd_epoch
label,epoch=data_predict(test_data_path,test_event_path,cut_begin,cut_end)

the current epoch: T 68
the current epoch: T 35
the current epoch: N 11
the current epoch: 0 20
the current epoch: 0 11
the current epoch: J 5
the current epoch: P 11
the current epoch: J 5
the current epoch: T 11


IndexError: ignored

In [ ]:
pd.DataFrame([label,epoch]).to_csv('S%d-result.csv'%s)

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#model = RandomForestClassifier(max_depth=2, random_state=0)
#model.fit(x_train,y_train)
#predict=model.predict(test_x)
#predict_prob=model.predict_proba(test_x)[:,1]
#feature_important=model.feature_importances_
#print('预测结果',predict)
#print('预测概率',predict_prob)
#print('特征重要程度',feature_important)